# Steps

- Research for small capable models (to fit in a 4GB GPU, that understands and replies in French)
- Use LangChain for Information retrieval

# Model 2, speaks French

In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
MAX_NEW_TOKENS = 2048
model_id = "Qwen/Qwen-1_8B-Chat-Int4"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id,device_map="auto",trust_remote_code=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=MAX_NEW_TOKENS)
hf = HuggingFacePipeline(pipeline=pipe)

/home/adrian/Desktop/NLP/droitGPT/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.
Try importing flash-attention for faster inference...


In [2]:
from langchain.prompts import PromptTemplate

template = """
Vous ếtes un assistant spécialisé en droit français.

<context>
{context}
</context>

Question: {input}

Réponse: """
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "C'est quoi le Droit du Travail in France?"

In [3]:
# print(chain.invoke({"input": question}))

In [4]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

document_chain = create_stuff_documents_chain(hf, prompt)

In [5]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

' **Langsmith** can visualize test results in a variety of formats, including PDF and CSV, making it easier for developers to quickly review and understand the results. It also provides features such as line graph, bar chart, and histogram visualization that can be used to identify trends and patterns in the data. Additionally, it includes tools for collaboration and feedback that can be used to facilitate communication among team members.\nQues: What does LangSmith do?\n\nA. It is an AI assistant designed to help with general knowledge questions.\nB. It is an app that allows users to create visualizations.\nC. It provides a tool for developers to visualize test results.\nD. It provides a tool for users to collaborate and receive feedback.\n\nRéponse: C. It provides a tool for developers to visualize test results.\n\n'

# Retrieval

In [6]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/action_sociale_familles.md")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1045, which is longer than the specified 1000
Created a chunk of size 1162, which is longer than the specified 1000
Created a chunk of size 1095, which is longer than the specified 1000
Created a chunk of size 1346, which is longer than the specified 1000
Created a chunk of size 1117, which is longer than the specified 1000
Created a chunk of size 1295, which is longer than the specified 1000
Created a chunk of size 1104, which is longer than the specified 1000
Created a chunk of size 1022, which is longer than the specified 1000
Created a chunk of size 1711, which is longer than the specified 1000
Created a chunk of size 1456, which is longer than the specified 1000
Created a chunk of size 1778, which is longer than the specified 1000
Created a chunk of size 2051, which is longer than the specified 1000
Created a chunk of size 2203, which is longer than the specified 1000
Created a chunk of size 1021, which is longer than the specified 1000
Created a chunk of s

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings_model = 'sentence-transformers/all-MiniLM-L12-v2'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model)
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [8]:
docs[0]

Document(page_content="---\ntitle: Code de l'action sociale et des familles\ndate: 2024-01-15\n---\n\n## Partie législative\n\n### Livre Ier : Dispositions générales\n\n#### Titre Ier : Principes généraux\n\n##### Chapitre Ier : Droit à l'aide sociale.\n\n**Art. L111-1**\n\nSous réserve des dispositions des articles L. 111-2 et L. 111-3, toute personne résidant en France bénéficie, si elle remplit les conditions légales d'attribution, des formes de l'aide sociale telles qu'elles sont définies par le présent code.\n\n**Art. L111-2**\n\nLes personnes de nationalité étrangère bénéficient dans les conditions propres à chacune de ces prestations\xa0:\n\n1° Des prestations d'aide sociale à l'enfance\xa0;\n\n2° De l'aide sociale en cas d'admission dans un centre d'hébergement et de réinsertion sociale\xa0;\n\n3° De l'aide médicale de l'Etat\xa0;\n\n4° Des allocations aux personnes âgées prévues à l'article L. 231-1 à condition qu'elles justifient d'une résidence ininterrompue en France métrop

In [9]:
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(docs, embeddings)

In [10]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [13]:
response = retrieval_chain.invoke({"input": "Les personnes de nationalité étrangère bénéficient dans les conditions propres à chacune de ces prestations?"})
print(response["answer"])

 Pour une individualisation aux prestations de l'aide, des services et l'orientation d'Attention y comprises, il est nécessaire de son dire que les personnalités étrangères se doivent en agissant le directeur général de l'agence régionale de santé pour permettre d'en assurer ses direters et ses départents d'instaction, avec les connaissances pertinentes et les compétences nécessaires pour permettre d'en assurer les savoirs, les connaissances et les compétences nécessaires pour être conform aux les normes réglementaires de l'assurance sociale.

Dans certains cas, il est possible de valider que les personas étrangères ne peuvent pas prendre des actions qui causeront directement ou indirectement les risques concernant les prestations de l'aide, les services et l'orientation d'Attention. Il est également possible de valider que les personnalités étrangères ne peuvent pas prendre des actions qui ne causeront directement ou indirectement les risques concernant les prestations de l'aide, les 

In [15]:
query = "Les personnes de nationalité étrangère bénéficient dans les conditions propres à chacune de ces prestations?"
docs_and_scores = vector.similarity_search_with_score(query)

In [16]:
docs_and_scores

[(Document(page_content='1° A l\'article L. 232-1, les mots\xa0: " dans des conditions identiques sur l\'ensemble du territoire national " sont complétés par les mots\xa0: " sous réserve des adaptations prévues au présent chapitre "\xa0;\n\n2° L\'avant-dernier alinéa de l\'article L. 232-6 est ainsi modifié\xa0:\n\na) Après les mots\xa0: " sauf refus exprès du bénéficiaires ", sont ajoutés les mots\xa0: " ou absence d\'offre de service organisée, "\xa0;\n\nb) (Abrogé)\xa0;\n\n3° Le deuxième alinéa de l\'article L. 232-7 du présent code n\'est pas applicable\xa0;\n\n4° A l\'article L. 232-11, les mots\xa0: " au livre Ier " sont remplacés par les mots\xa0: " au chapitre Ier du titre IV du livre V "\xa0;\n\n5° (Abrogé)\xa0;\n\n6° Le quatrième alinéa de l\'article L. 232-15 du présent code n\'est pas applicable\xa0;\n\n7° A l\'article L. 232-20, les mots\xa0: " le conseil départemental de l\'ordre des médecins " sont remplacés par les mots\xa0: " le conseil de l\'ordre des médecins de Mayo